In [2]:
import os 
import sys

import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from scipy.io import wavfile as wav
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
PATH = os.getcwd() + '/speech_commands_v0.01/'

In [4]:
def get_directory_contents(path):
    return os.listdir(path)

def open_file(filename):
    
    f = open(filename)
    return f.read().splitlines()

def compile_dataset(folders):
    
    total_words = []
    for folder in folders:
        words = get_directory_contents(path=PATH+folder)
        words = [folder+'/'+word for word in words]
        total_words = total_words + words
    
    dataset = create_df(words=total_words)
                           
    return dataset

def create_df(words):
    
    data = pd.DataFrame({'recordings':words})

    data['word'] = data['recordings'].str.split('/').str[0]
    data['speaker_id'] = data['recordings'].str.split('/').str[1]
    data['speaker_id'] =data['speaker_id'].str.split('_').str[0]
    
    return data

def summary(data):
    
    summary_df = pd.DataFrame()
    summary_df['total_recordings'] = [data.shape[0]]
    summary_df['total_speakers'] = len(data['speaker_id'].unique())
    summary_df['total_words'] = len(data['word'].unique())
    
    return summary_df

def word_distribution():
    
    word_count = data['word'].value_counts()
    ax = word_count.plot(kind='bar', figsize=(8,4), alpha=0.5)
    plt.show()

In [5]:
remove_words = [
    '.DS_Store', 'validation_list.txt', 'LICENSE',
    '_background_noise_', 'README.md', 'testing_list.txt'
]

names = get_directory_contents(path=PATH)
names = [word for word in names if word not in remove_words]

In [6]:
val_list = open_file(filename=PATH+'validation_list.txt')
val_df = create_df(words=val_list)
test_list = open_file(filename=PATH+'testing_list.txt')
test_df = create_df(words=test_list)
total_df = compile_dataset(folders=names)

training_df = total_df[~total_df['recordings'].isin(val_df['recordings'])]
training_df = training_df[~training_df['recordings'].isin(test_df['recordings'])]

In [7]:
fn = 'speech_commands_v0.01/four/10c6d873_nohash_0.wav'
librosa_audio, librosa_sample_rate = librosa.load(fn)
scipy_sample_rate, scipy_audio = wav.read(fn)
print("Original sample rate: {}".format(scipy_sample_rate))
print("Librosa sample rate: {}".format(librosa_sample_rate))

Original sample rate: 16000
Librosa sample rate: 22050


In [8]:
print('Original audio file min~max range: {} to {}'.format(np.min(scipy_audio), np.max(scipy_audio)))
print('Librosa audio file min~max range: {0:.2f} to {0:.2f}'.format(np.min(librosa_audio), np.max(librosa_audio)))

Original audio file min~max range: -32768 to 16883
Librosa audio file min~max range: -1.00 to -1.00


In [9]:
mfccs = librosa.feature.mfcc(y=librosa_audio, sr=librosa_sample_rate, n_mfcc = 40)

In [11]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs_processed

In [12]:
def dataset_prep(df):
    features = []
    # Iterate through each sound file and extract the features 
    for index, row in df.iterrows():
        file_name = 'speech_commands_v0.01/'+row['recordings']
        class_label = row["word"]
        data = extract_features(file_name)

        features.append([data, class_label])
    # Convert into a Panda dataframe 
    featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
    return featuresdf

In [15]:
featuresdf = dataset_prep(df=training_df)
featuresdf.to_csv("training_set.csv")

testdf = dataset_prep(df=test_df)
valdf = dataset_prep(df=val_df)

In [16]:
testdf.to_csv("testdf.csv")
valdf.to_csv("validation_set.csv")

In [17]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [18]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

x_val = np.array(valdf.feature.tolist())
y_val = np.array(valdf.class_label.tolist())
y_val = to_categorical(le.transform(y_val))

x_test = np.array(testdf.feature.tolist())
y_test = np.array(testdf.class_label.tolist())
y_test = to_categorical(le.transform(y_test))

In [19]:
num_labels = yy.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(30))
    model.add(Activation('relu'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model
model = build_model_graph()

In [20]:
num_epochs = 100
num_batch_size = 32
model.fit(X, yy, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_val, y_val), verbose=1)

Epoch 1/100
1597/1597 [==============================] - 2s 1ms/step - loss: 7.2422 - accuracy: 0.0560 - val_loss: 6.4439 - val_accuracy: 0.1091
Epoch 2/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.4780 - accuracy: 0.1159 - val_loss: 6.3437 - val_accuracy: 0.1478
Epoch 3/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.4240 - accuracy: 0.1387 - val_loss: 6.2718 - val_accuracy: 0.1586
Epoch 4/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.1035 - accuracy: 0.1480 - val_loss: 5.9083 - val_accuracy: 0.1606
Epoch 5/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.8574 - accuracy: 0.1546 - val_loss: 5.8876 - val_accuracy: 0.1590
Epoch 6/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.8908 - accuracy: 0.1639 - val_loss: 5.8880 - val_accuracy: 0.1709
Epoch 7/100
1597/1597 [==============================] - 2s 988us/step - loss: 5.8915 - accuracy: 0.1734 - val_loss: 5.8768 - val_

1597/1597 [==============================] - 2s 1ms/step - loss: 5.7961 - accuracy: 0.1991 - val_loss: 5.8642 - val_accuracy: 0.2045
Epoch 58/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.8131 - accuracy: 0.2046 - val_loss: 5.8655 - val_accuracy: 0.2107
Epoch 59/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.8000 - accuracy: 0.2027 - val_loss: 5.8587 - val_accuracy: 0.2114
Epoch 60/100
1597/1597 [==============================] - 2s 977us/step - loss: 5.8462 - accuracy: 0.2042 - val_loss: 5.8835 - val_accuracy: 0.2071
Epoch 61/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.8199 - accuracy: 0.2050 - val_loss: 5.8766 - val_accuracy: 0.2118
Epoch 62/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.8008 - accuracy: 0.2058 - val_loss: 5.8400 - val_accuracy: 0.2046
Epoch 63/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.7581 - accuracy: 0.2056 - val_loss: 5.8680 - val_accura

In [21]:
score = model.evaluate(X, yy, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_val, y_val, verbose=0)
print("Validation Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 19.96%
Validation Accuracy: 19.95%
Testing Accuracy: 18.95%


### Normal Machine Learning model

In [22]:
clf = LogisticRegression(multi_class='multinomial', solver='newton-cg')

In [33]:
def to_category(columns, dataframe):
    """Convert a list of columns, from a dataframe, to a category datatype"""
    for column in columns: 
        dataframe[column] = dataframe[column].astype('category')
    return dataframe
columns=['word']

training_df = to_category(columns=columns, dataframe=training_df)
test_df = to_category(columns=columns, dataframe=test_df)
val_df = to_category(columns=columns, dataframe=val_df)

In [38]:
clf.fit(X, training_df['word'])

print("Training Accuracy: {0:.2%}".format(clf.score(X, training_df['word'])))
print("Validation Accuracy: {0:.2%}".format(clf.score(x_val, val_df['word'])))
print("Testing Accuracy: {0:.2%}".format(clf.score(x_test, test_df['word'])))

/Users/harshinderchadha/general/lib/python3.8/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


Training Accuracy: 29.40%
Validation Accuracy: 29.07%
Testing Accuracy: 26.53%
